## Import some python libraries

In [1]:
%matplotlib inline
import pickle
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random
import tensorflow.keras.utils
import tensorflow.keras.models as models
from tensorflow.keras.layers import Reshape,Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import *
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
import joblib  # For saving and loading model
import time

## Load the data

In [2]:
Xd = pickle.load(open("data/RML2016.10a_dict.pkl",'rb'),encoding = "bytes")
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

## Split the data into training and test sets

In [3]:
# Partition the data
#  into training and test sets of the form we can train/test on 
np.random.seed(2020)
n_examples = X.shape[0]
n_train = n_examples // 2
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]

## Convert the dependent variables (labels) into one-hoe encoding format

In [4]:
#one-hot encoding the label
lb = preprocessing.LabelBinarizer()
lb.fit(np.asarray(lbl)[:,0])
print(lb.classes_)
lbl_encoded=lb.transform(np.asarray(lbl)[:,0])
y_train=lbl_encoded[train_idx]
y_test=lbl_encoded[test_idx]

[b'8PSK' b'AM-DSB' b'AM-SSB' b'BPSK' b'CPFSK' b'GFSK' b'PAM4' b'QAM16'
 b'QAM64' b'QPSK' b'WBFM']


In [5]:
in_shp = list(X_train.shape[1:])
print(X_train.shape, in_shp)
classes = mods

(110000, 2, 128) [2, 128]


## Check the maximum and minimum values in both training and test sets

In [6]:
np.max(X_train), np.min(X_train)

(0.16046731, -0.15494554)

In [7]:
np.max(X_test), np.min(X_test)

(0.16422912, -0.15118897)

## Flateen the 2D input into 1D vector

In [8]:
X_train = X_train.reshape((X_train.shape[0], -1))

In [9]:
X_test = X_test.reshape((X_test.shape[0], -1))

## Performing principal component analysis (PCA)

In [10]:
# P_components = 100

# pca = PCA(n_components = P_components, whiten = True)

# X_train_pca = pca.fit_transform(X_train)
# X_test_pca = pca.fit_transform(X_test)

## Build the ANN model

In [11]:
# model = models.Sequential()

# # model.add(Dense(64, input_shape = (X_train_pca.shape[1],), activation = 'tanh'))
# model.add(Dense(64, input_shape = (X_train.shape[1],), activation = 'relu'))
# model.add(Dense(32, activation = 'relu'))
# model.add(Dense(16, activation = 'relu'))
# model.add(Dense(len(classes)))
# model.add(Activation('softmax'))
# model.summary()

In [12]:
model = models.Sequential()

# model.add(Dense(64, input_shape = (X_train_pca.shape[1],), activation = 'tanh'))
model.add(Dense(64, input_shape = (X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(len(classes)))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                5

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# model.fit(X_train_pca, y_train, batch_size = 128, steps_per_epoch = 110000//128, epochs = 5, verbose = 1,
#           validation_data = (X_test_pca, y_test), validation_batch_size = 128, validation_steps = 110000//128)
start = time.time()
model.fit(X_train, y_train, batch_size = 128, steps_per_epoch = 110000//128, epochs = 5, verbose = 1,
          validation_data = (X_test, y_test), validation_batch_size = 128, validation_steps = 110000//128)
end = time.time()
training_time = end - start

print ("The training time is %.3f seconds" %(training_time))

Epoch 1/5
859/859 [==============================] - 8s 7ms/step - loss: 2.2219 - accuracy: 0.1848 - val_loss: 1.8946 - val_accuracy: 0.2844
Epoch 2/5
859/859 [==============================] - 3s 4ms/step - loss: 1.8108 - accuracy: 0.3210 - val_loss: 1.7836 - val_accuracy: 0.3150
Epoch 3/5
859/859 [==============================] - 3s 4ms/step - loss: 1.6999 - accuracy: 0.3522 - val_loss: 1.7291 - val_accuracy: 0.3358
Epoch 4/5
859/859 [==============================] - 3s 4ms/step - loss: 1.6370 - accuracy: 0.3732 - val_loss: 1.7477 - val_accuracy: 0.3307
Epoch 5/5
859/859 [==============================] - 3s 4ms/step - loss: 1.6010 - accuracy: 0.3864 - val_loss: 1.6835 - val_accuracy: 0.3542
The training time is 21.501 seconds


In [15]:
score = model.evaluate(X_test, y_test)
# score = model.evaluate(X_test_pca, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

3438/3438 [==============================] - 4s 1ms/step - loss: 1.6830 - accuracy: 0.3544
Test loss: 1.683009386062622
Test accuracy: 0.3544090986251831
